In [13]:
import os
import cv2
import json
import glob
import numpy as np
import skimage.io as io

#save all image files and json files separately to list
image_list = sorted(glob.glob('Images/Head/head_training_set/hc_raw/*.png'))
ann_list = sorted(glob.glob('Images/Head/head_training_set/HC_Mask_JSON/*.json'))
save_path = "Images/Head/head_training_set/hc_binary"

#create blank mask with image sizes
def create_binary_masks(im, shape_dicts):
    
    blank = np.zeros(shape=(im.shape[0], im.shape[1]), dtype=np.float32)
    for shape in shape_dicts:
        points = np.array(shape['points'], dtype=np.int32)
        cv2.fillPoly(blank, [points], 255)
    return blank

#get annotated points
def get_poly(ann_path):
    
    with open(ann_path) as handle:
        data = json.load(handle)
    shape_dicts = data['shapes']
    
    return shape_dicts

#iterate every image and its json file to create binary mask
for im_fn, ann_fn in zip(image_list, ann_list):
    
    im = cv2.imread(im_fn, 0)
    shape_dicts = get_poly(ann_fn)
    im_binary = create_binary_masks(im, shape_dicts)
    
    #extract the name of image file
    filename = im_fn.split('.png')[-2].split('\\')[-1] + '.png'
    io.imsave(os.path.join(save_path, filename), im_binary)
    cv2.imwrite(filename, im_binary)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    